In [52]:
!nvidia-smi

Sat Nov 30 18:47:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     Off |   00000000:81:00.0 Off |                    0 |
|  0%   47C    P0             81W /  300W |   44564MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import sys, getopt
import jsonlines
import random
import numpy as np
import pickle
from tqdm import tqdm
import pandas as pd
import re
import numpy as np
import os
import time
from openai import OpenAI
import pandas as pd
import jsonlines
import numpy as np
from tqdm import tqdm
import json
import tiktoken
import ollama
import csv
import copy
encoding = tiktoken.encoding_for_model("gpt-4")
import os
import sys
import logging
from tqdm import tqdm

# CHAGNE PROMPT HEREEEEEEE
with open('./prompts/prompt1-8.txt', 'r') as file:
    sys_prompt = file.read()

modelfile=f'''
FROM llama3.1:70b
system """
{sys_prompt.strip()}
"""
'''
model = 'myexample1'
num_ctx = 17000 # 15000 original, 
ollama.create(model='myexample1', modelfile=modelfile)



{'status': 'success'}

In [57]:

###################
# Utility Functions
###################

def getWithoutAC(s):
    s = s[1:-1]
    s = s.split(", ")
    withoutAccess = []
    for k in s:
        if "[AC]" not in k:
            withoutAccess.append(k)
    return "<|>".join(withoutAccess)

def extractAC(s):
    s = s[1:-1]
    s = s.split(", ")
    ans = -1
    
    for k in s:
        if "[AC]" in k:
            match = re.search(r'\((\d+)\)', k)
            if match:
                number = int(match.group(1))
                ans = max(ans, number)
    m = {-1: 0, 0: 0, 1: 0, 2: 1, 3: 1}
    return m[ans]

def parse_data_string(dict_str):
    newDict = {}
    dict_str = dict_str[1:-1]
    dict_str = dict_str.split(',')
    for d in dict_str:
        newDict[d.split('=')[0].strip()] = d.split('=')[1].strip()
    return newDict

def getData(fileName):
# Read the file
    with open(fileName, "r") as file:
        lines = file.readlines()

    # Initialize variables
    current_ep = None
    all_entries = []

    # Iterate over the lines in the file
    for line in lines:
        # If the line starts with 'EP:', it's the start of a new EP block
        if line.startswith('EP:'):
            current_ep = line.strip().split('EP: ')[1]
        # If the line starts with 'ControlFlow:', it's the start of a new data block
        elif line.startswith('ControlFlow:'):
            current_block = {}
            current_block['EP'] = current_ep
            current_block['code'] = line.strip().split('ControlFlow: ')[1]
            all_entries.append(current_block)
        # elif line.startswith('Features:'):
        #     newDict = parse_data_string(line.strip().split('Features: ')[1])
        #     for k in newDict.keys():
        #         current_block[k] = newDict[k]
        #     all_entries.append(current_block)

    # Convert the list of blocks to a dataframe
    df = pd.DataFrame(all_entries)
    df["label"] = df["code"].apply(extractAC)
    df["code"] = df["code"].apply(getWithoutAC)
    return df

import numpy as np

def slidingWindow(path):
    window = []
    curPath = []
    for p in path:
        curPath.append(p)
        window.append(list(curPath))
    return window


def decompose(df):
    # Create a helper column for chunk grouping
    df['group'] = np.where(df['label'] == 0, np.arange(len(df)), df.groupby('EP').cumcount() // 4)

    # Group by 'EP', 'label' and 'group', and then merge 'subsequences'
    df_agg = df.groupby(['EP', 'label', 'group']).agg({
        'code': lambda x: '<PATH_SEP>'.join(x)
    })

    # Reset the index to get 'EP' and 'label' as columns
    df_agg.reset_index(inplace=True)

    # Drop the 'group' column
    df_agg.drop(columns=['group'], inplace=True)

    # If you still want to aggregate 'label' by taking the max value
    df_agg['label'] = df_agg.groupby('EP')['label'].transform('max')

    return df_agg



def shuffle_dataframes(df):
    # Concatenate the dataframes
    
    # Separate the labels from the features
    X = df.drop('label', axis=1)
    y = df['label']

    # Split the data into train and remaining data (test + validation)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Create train, test, and validation dataframes
    train_df = pd.concat([X_train, y_train], axis=1)
    test_df = pd.concat([X_test, y_test], axis=1)
    
    return train_df, test_df


# Creating the Training Data for the model
def createJsonL(df, fileName):
    fileName = fileName.replace(".txt", "")
    json_objects = []

    # Group the DataFrame by unique values in the 'EP' column
    grouped = df.groupby('EP')

    # Iterate over each group
    for _, group in grouped:
        # Shuffle the group randomly
        shuffled_group = group.sample(frac=1, random_state=42)  # Set a random_state for reproducibility

        # Keep at most 10 rows in the shuffled group
        # shuffled_group = shuffled_group.head(10)

        # Iterate over each row in the shuffled group
        for _, row in shuffled_group.iterrows():
            prompt = []

            # Iterate over each column (excluding 'label')
            for column in shuffled_group.columns:
                if column != 'label':
                    prompt.append(str(column) + ' = ' + str(row[column]))

            # Create the JSON object
            json_object = {
                'code': " ".join(prompt),
                'label': int(row['label'])
            }

            # Append the JSON object to the list
            json_objects.append(json_object)

    with jsonlines.open(fileName, 'w') as writer:
        writer.write_all(json_objects)


In [58]:
file_name = "data/aosp.txt"
try:
    df = getData(file_name)
except:
    print(f"Could not read {file_name}")

label_counts = df['label'].value_counts()

print("################\n1 - Protection Required\n0 - No Protection\n################\n")

print("LabelCounts:\n", label_counts)
merged_aaAdf = decompose(df)
merged_df = merged_aaAdf
trainDF, testDF = shuffle_dataframes(merged_df)

label_counts = merged_df['label'].value_counts()
print("LabelCounts:\n", label_counts)

print(merged_df.head())

createJsonL(trainDF, f'{file_name}_train.jsonl')
createJsonL(testDF, f'{file_name}_test.jsonl')

################
1 - Protection Required
0 - No Protection
################

LabelCounts:
 label
1    36207
0     9623
Name: count, dtype: int64
LabelCounts:
 label
1    14903
0     5242
Name: count, dtype: int64
                                 EP  label  \
0  abandonAudioFocus_AudioService_5      0   
1  abandonAudioFocus_AudioService_5      0   
2  abandonAudioFocus_AudioService_5      0   
3  abandonAudioFocus_AudioService_5      0   
4  abandonAudioFocus_AudioService_5      0   

                                                code  
0  [get]: MediaFocusControl var6 = this.mMediaFoc...  
1  [get]: MediaFocusControl var6 = this.mMediaFoc...  
2  [get]: MediaFocusControl var6 = this.mMediaFoc...  
3  [get]: MediaFocusControl var6 = this.mMediaFoc...  
4  [get]: MediaFocusControl var6 = this.mMediaFoc...  


In [64]:
# trainDF.iloc[0]['code']
def generate_java_method(name, code):
    # Extract the API name and method name dynamically
    api_name, method_name = name.split("_", 1)

    # Start the Java code formatting
    java_code = []
    java_code.append(f"// API: {api_name}")
    java_code.append(f"// Service: {method_name}\n")
    java_code.append(f"public void {api_name}() {{")

    # Split and clean up the code
    lines = code.split("<|>")
    for line in lines:
        # Remove [..]: and clean up the lines
        cleaned_line = re.sub(r"\[.*?\]: ", "", line).strip()
        if cleaned_line:
            # Ensure only one semicolon is added
            if not cleaned_line.endswith(";"):
                cleaned_line += ";"
            java_code.append(f"    {cleaned_line}")

    java_code.append("}")
    return "\n".join(java_code)


# s= generate_java_method(trainDF.iloc[0]['EP'],trainDF.iloc[0]['code'])
# s
trainDF['java_code'] = trainDF.apply(lambda row: generate_java_method(row['EP'], row['code']), axis=1)
## DROP THE DUPLICATES USING LENGTH
trainDF['code_length'] = trainDF['java_code'].map(len)
trainDF = (
    trainDF.sort_values(by='code_length', ascending=False)  
    .drop_duplicates(subset=trainDF.columns[0])             
    .reset_index(drop=True)                                 
)
trainDF = trainDF.drop(columns=['code_length'])
trainDF['service_name'] = trainDF['EP'].apply(lambda word: word.split("_")[1])


In [67]:

def run_first_prompt_Ollama(method_code,run):
    """ runs the first prompt - extract sinks from the traces
    """
    
    global sys_prompt

    # user_prompt = get_method_traces_from_file(file_path, interface, method)
    
    user_prompt = method_code
    if(not run):
        return {
            "system_message": sys_prompt,
            "user_message": user_prompt,
            "response": "Not running"
        }
        
        
    
    response = ollama.chat(model=model, messages=[
    {
        'role': 'user',
        'content': user_prompt,
    },
    ]
    ,
     options={
        'num_ctx': num_ctx,
        # 'temperature': 0.3 # CHECK TEMPERATURE HERERERERERERERER
    }
    
    )
    
    # logging.info(f"Response for {method} = {response['message']['content']}")

    
    return {
        "system_message": sys_prompt,
        "user_message": user_prompt,
        "response": response['message']['content']
    }
    

# get the json from the res["response"]

# def extract_json_from_string(input_string):
#     """
#     Extracts JSON from the given string.

#     Args:
#         input_string (str): The string containing embedded JSON.

#     Returns:
#         dict: The extracted JSON as a Python dictionary.
#     """
#     try:
#         # Use a regex pattern to extract the JSON part
#         json_pattern = r"```(?:json)?\n(.*?)\n```"
#         match = re.search(json_pattern, input_string, re.DOTALL)
#         if match:
#             json_string = match.group(1)
#             return json.loads(json_string)
#         else:
#             raise ValueError("No JSON found in the provided string.")
#     except json.JSONDecodeError as e:
#         raise ValueError(f"Error decoding JSON: {e}")

def extract_json_from_string(input_string):
    valid_jsons = []
    
    try:
        # Use a regex pattern to extract all JSON parts embedded between ```json``` blocks
        json_pattern = r"```(?:json)?\s*(.*?)\s*```"
        matches = re.findall(json_pattern, input_string, re.DOTALL)
        
        # Process each match (potential JSON string)
        for json_string in matches:
            try:
                # Try to load the string as JSON
                json_obj = json.loads(json_string)
                valid_jsons.append(json_obj)
            except json.JSONDecodeError as e:
                continue
                # Print the invalid JSON for debugging
                # print(f"Invalid JSON: {json_string}")
                # print(f"Error: {e}")
        
        # Return all valid JSON objects found
        if not valid_jsons:
            print(input_string)
            # raise ValueError("No valid JSON found in the provided string.")
            raise Exception("No valid JSON found in the provided string.")
        
        return valid_jsons[0]
    
    except Exception as e:
        print(f"Error processing string: {e}")
        
        return []

In [ ]:


AMS_Df = trainDF[trainDF['service_name'] == 'ActivityManagerService'].copy()
AMS_Df.shape
AMS_Df['json_answer'] = None


output_folder = "./output/week2/output3"
counter = 0
for index, row in tqdm(AMS_Df.iterrows(), total=AMS_Df.shape[0], desc="Processing rows"):
    # print(row['EP'])
    # print(row['service_name'])
    res = run_first_prompt_Ollama(row['java_code'],True)
    json_answer = None
    try:
        json_answer = extract_json_from_string(res["response"])
    except Exception as e:
        print(f"Error extracting JSON from response: {e}")
        print(row['EP'])
        print(row['service_name'])
        # continue
        # Store the json_answer in the new column
    AMS_Df.at[index, 'json_answer'] = json_answer

    # Write response details to files
    folder_path = f'{output_folder}/{row["EP"].split("_")[0]}'
    os.makedirs(folder_path, exist_ok=True)

    with open(f'{folder_path}/system_message.txt', 'w') as f:
        f.write(res["system_message"])
    with open(f'{folder_path}/user_message.txt', 'w') as f:
        f.write(res["user_message"])
    with open(f'{folder_path}/response.txt', 'w') as f:
        f.write(res["response"])

pickle_file_path = "./serialized_data/week2/AMS_Df_3.pkl"

# Serialize the DataFrame using pickle
with open(pickle_file_path, 'wb') as file:
    pickle.dump(AMS_Df, file)

print(f"DataFrame serialized and saved to {pickle_file_path}")

Processing rows:  12%|█▏        | 23/192 [30:25<3:19:59, 71.00s/it]

Error processing string: No valid JSON found in the provided string.


Processing rows:  38%|███▊      | 72/192 [1:25:53<2:18:06, 69.06s/it]

Error processing string: No valid JSON found in the provided string.


Processing rows:  44%|████▍     | 84/192 [1:38:31<1:43:18, 57.39s/it]

...

Error processing string: No valid JSON found in the provided string.


Processing rows:  97%|█████████▋| 187/192 [3:05:41<02:18, 27.75s/it]

Error processing string: No valid JSON found in the provided string.


Processing rows: 100%|██████████| 192/192 [3:07:57<00:00, 58.74s/it]

DataFrame serialized and saved to ./serialized_data/week2/AMS_Df_2.pkl


In [25]:
# 192 rows × 6 columns

# check which ones are None
AMS_Df = AMS_Df[AMS_Df["json_answer"].isnull()]
# [AMS_Df["json_answer"]]
AMS_Df

,EP,code,label,java_code,service_name,json_answer
24,stopService_ActivityManagerService_5,[inv]: boolean var10 = hasFileDescriptors();<|...,1,// Method: stopService\n// Service: ActivityMa...,ActivityManagerService,None
35,getProcessPss_ActivityManagerService_2,[inv]: int var6 = Binder.getCallingPid();<|>[i...,1,// Method: getProcessPss\n// Service: Activity...,ActivityManagerService,None
37,setServiceForeground_ActivityManagerService_7,[inv]: ActivityManagerService.boostPriorityFor...,1,// Method: setServiceForeground\n// Service: A...,ActivityManagerService,None
84,handleApplicationCrash_ActivityManagerService_3,[inv]: ProcessRecord var6 = findAppProcess(par...,1,// Method: handleApplicationCrash\n// Service:...,ActivityManagerService,None
141,setFocusedStack_ActivityManagerService_2,[get]: ActivityTaskManagerService var3 = this....,1,// Method: setFocusedStack\n// Service: Activi...,ActivityManagerService,None
...,...,...,...,...,...,...
2186,registerUserSwitchObserver_ActivityManagerServ...,[get]: UserController var4 = this.mUserControl...,1,// Method: registerUserSwitchObserver\n// Serv...,ActivityManagerService,None
2198,noteWakeupAlarm_ActivityManagerService_6,[inv]: boolean var9 = isEmpty();<|>[ins]: 11 =...,0,// Method: noteWakeupAlarm\n// Service: Activi...,ActivityManagerService,None
2199,noteAlarmFinish_ActivityManagerService_5,[inv]: boolean var8 = isEmpty();<|>[ins]: 10 =...,0,// Method: noteAlarmFinish\n// Service: Activi...,ActivityManagerService,None
2203,noteAlarmStart_ActivityManagerService_5,[inv]: boolean var8 = isEmpty();<|>[ins]: 10 =...,0,// Method: noteAlarmStart\n// Service: Activit...,ActivityManagerService,None


In [9]:
# return {
#         "system_message": prompt1,
#         "user_message": user_prompt,
#         "response": response['message']['content']
#     }
# write res["response"] to a txt file
with open('output3.txt', 'w') as f:
    f.write(res["response"])
    f.close()

In [12]:
trainDF

word = "startRecentsActivity_ActivityTaskManagerService_4"
service = word.split("_")[1]
service 

'ActivityTaskManagerService'

In [16]:
trainDF.columns

Index(['EP', 'code', 'label', 'java_code', 'service_name'], dtype='object')

In [14]:
res["response"]

'After analyzing the provided Java code, I have identified several potential sinks that may require access control. Here are the results in JSON format:\n\n```\n{\n  "Sinks": [\n    [\n      "PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();",\n      "ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);"\n    ],\n    [\n      "IntentFirewall var421 = var420.mIntentFirewall;",\n      "boolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);"\n    ],\n    [\n      "AppOpsService var439 = var438.mAppOpsService;"\n    ]\n  ]\n}\n```\n\nRanking the sinks from most sensitive to least sensitive, I would place them in the following order:\n\n1. The first sink involves accessing the PackageManagerInternal instance and resolving a service using the resolveService method. This could potentially allow an attacker to access sensitive information about installed services or even install malicious services.\n2. The second sink 

In [ ]:




json_answer = extract_json_from_string(res["response"])

In [21]:
json_answer

{'Sinks': [['PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();',
   'ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);'],
  ['IntentFirewall var421 = var420.mIntentFirewall;',
   'boolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);'],
  ['AppOpsService var439 = var438.mAppOpsService;']]}

In [64]:
# read the serialized DataFrame
with open(pickle_file_path, 'rb') as file:
    AMS_Df = pickle.load(file)
    
AMS_Df.head()

,EP,code,label,java_code,service_name,json_answer
0,bindService_ActivityManagerService_9,"[inv]: int var12 = bindIsolatedService(param2,...",1,// Method: bindService\n// Service: ActivityMa...,ActivityManagerService,{'Sinks': [['ActivityManagerService var101 = t...
2,sendIdleJobTrigger_ActivityManagerService_1,[inv]: long var10 = Binder.clearCallingIdentit...,1,// Method: sendIdleJobTrigger\n// Service: Act...,ActivityManagerService,"{'Sinks': [['int var22 = broadcastIntent(0, va..."
7,startService_ActivityManagerService_7,[inv]: boolean var12 = hasFileDescriptors();<|...,1,// Method: startService\n// Service: ActivityM...,ActivityManagerService,{'Sinks': [['boolean var425 = var421.checkServ...
10,updateConfiguration_ActivityManagerService_2,[get]: ActivityTaskManagerService var3 = this....,1,// Method: updateConfiguration\n// Service: Ac...,ActivityManagerService,{'Sinks': [['SystemProperties.set(persist.sys....
22,getProcessMemoryInfo_ActivityManagerService_2,[inv]: long var6 = SystemClock.uptimeMillis();...,1,// Method: getProcessMemoryInfo\n// Service: A...,ActivityManagerService,"{'Sinks': [['Debug.getMemoryInfo(var60,var58);..."
